In [1]:
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from google.cloud import storage
#from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

torch.cuda.empty_cache() 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [2]:
model = models.resnet50(pretrained=True)

In [3]:
os.getcwd()

'/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals'

In [4]:
## Creating a train dataset class
class ItemsTrainDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)

    def _load_examples(self):
        class_names = os.listdir('pytorch_images_tv_split_2/train')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('pytorch_images_tv_split_2/train',cl_name))
            example_fp = [os.path.join('pytorch_images_tv_split_2/train', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)
        #features = self.rgbify(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [5]:
## Creates a validation dataset class
class ItemsValDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)!=1 else x)

    def _load_examples(self):
        class_names = os.listdir('pytorch_images_tv_split_2/val')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}
        examples_list = []
        
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('pytorch_images_tv_split_2/val',cl_name))
            example_fp = [os.path.join('pytorch_images_tv_split_2/val', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)
        #features = self.rgbify(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [6]:
traindataset = ItemsTrainDataSet()

In [7]:
len(traindataset)

2132

In [8]:
valdataset = ItemsValDataSet()
len(valdataset)

922

In [9]:
## Created a classifier based on the RESNET50 pretrained model

class ItemClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,13)
  
    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [10]:
def train(model,traindataloader, valdataloader, epochs):
    optimiser = torch.optim.SGD(model.parameters(), lr=0.01)
    model_path = str(os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S")))   
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))
    batch_idx = 0
    
    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        model.to(device)
        model.train()
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))
        for inputs, labels in traindataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            #print(predictions.shape)
            #print(labels.shape)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            loss.backward()
            optimiser.step()
            model_save_dir = str(os.path.join(model_path, 'weights', epoch_combo, 'weights.pth'))
            full_path = str('/home/jupyter/facebook-marketplaces-recommendation-ranking-system/Practicals')
            #print(model_save_dir)
            #torch.save({'epoch': epoch,
            #    'model_state_dict': model.state_dict(),
            #    'optimizer_state_dict': optimiser.state_dict()}, 
                  #str(os.path.join(full_path, model_save_dir)))
            #           model_save_dir)
            torch.save({'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimiser.state_dict()},
                  str(os.path.join(full_path, model_save_dir)))

            optimiser.zero_grad()
            batch_idx += 1
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]
        training_loss /= len(traindataloader.dataset)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        for inputs, labels in valdataloader:
            #labels = labels.unsqueeze(1)
            #labels = labels.float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            predictions = model(inputs)
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(predictions, labels)
            validation_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim =1), dim=1)[1], labels)
            val_num_correct += torch.sum(correct).item()
            val_num_examples += correct.shape[0]
        validation_loss /= len(valdataloader.dataset)
        perf_dict = {}
        perf_dict[epoch] = {'training_loss': training_loss,
                            'val_loss': validation_loss,
                            'training_accuracy': tr_num_correct / tr_num_examples,
                            'val_accuracy': val_num_correct / val_num_examples}
                            
                            
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f},val_accuracy = {:.2f} '.format(epoch, training_loss, validation_loss, tr_num_correct / tr_num_examples,
                                                                                                                             val_num_correct / val_num_examples))

In [12]:
classifier = ItemClassifier()

Using cache found in /home/jupyter/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [12]:
'''
## unfreeze last two layers
for param in classifier.resnet50.layer3:
  param.requires_grad=True

for param in classifier.resnet50.layer4:
  param.requires_grad=True
'''

'\n## unfreeze last two layers\nfor param in classifier.resnet50.layer3:\n  param.requires_grad=True\n\nfor param in classifier.resnet50.layer4:\n  param.requires_grad=True\n'

In [13]:
'''
## define the layers to unfreeze and then retrain
layers_to_unfreeze = ['layers.2', 'layers.3']

for name, param in classifier.resnet50.named_parameters():
    for layer_name in layers_to_unfreeze:
        if layer_name in name:
            param.requires_grad = True
            break
'''

"\n## define the layers to unfreeze and then retrain\nlayers_to_unfreeze = ['layers.2', 'layers.3']\n\nfor name, param in classifier.resnet50.named_parameters():\n    for layer_name in layers_to_unfreeze:\n        if layer_name in name:\n            param.requires_grad = True\n            break\n"

In [ ]:
#train_dataset = ItemsTrainDataSet()
#val_dataset = ItemsValDataSet()
train_loader = DataLoader(dataset = traindataset, batch_size=16)
val_loader = DataLoader(dataset = valdataset, batch_size=16)
train(classifier, traindataloader= train_loader, valdataloader= val_loader, epochs=50)


Epoch: 0, Training Loss: 2.57, Validation Loss: 2.56, train_accuracy = 0.10,val_accuracy = 0.12 
Epoch: 2, Training Loss: 2.55, Validation Loss: 2.55, train_accuracy = 0.12,val_accuracy = 0.13 
Epoch: 3, Training Loss: 2.53, Validation Loss: 2.55, train_accuracy = 0.14,val_accuracy = 0.13 
Epoch: 4, Training Loss: 2.52, Validation Loss: 2.54, train_accuracy = 0.15,val_accuracy = 0.13 
Epoch: 5, Training Loss: 2.50, Validation Loss: 2.54, train_accuracy = 0.16,val_accuracy = 0.13 
Epoch: 6, Training Loss: 2.49, Validation Loss: 2.54, train_accuracy = 0.18,val_accuracy = 0.12 
Epoch: 7, Training Loss: 2.48, Validation Loss: 2.53, train_accuracy = 0.19,val_accuracy = 0.12 
Epoch: 8, Training Loss: 2.47, Validation Loss: 2.53, train_accuracy = 0.19,val_accuracy = 0.12 
Epoch: 9, Training Loss: 2.45, Validation Loss: 2.53, train_accuracy = 0.20,val_accuracy = 0.12 
Epoch: 11, Training Loss: 2.43, Validation Loss: 2.53, train_accuracy = 0.22,val_accuracy = 0.13 
Epoch: 12, Training Loss: 2.4

In [ ]:
train_loader.dataset.

In [ ]:
train_dataset[2][0].shape

In [ ]:
os.getcwd()

In [ ]:
for i in range(len(valdataset)):
    print(valdataset[i][0].shape)